<a href="https://colab.research.google.com/github/Lee-Kiwon/Quant_project/blob/main/US_Treasury_bond_Yield_Curve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 미국 국채 금리 커브 구현
# 출처 : Antifragile Domain- 퀀트 대디 https://m.blog.naver.com/quantdaddy/221759675102

In [ ]:
pip install QuantLib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 57.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import datetime
from bs4 import BeautifulSoup
import QuantLib as ql

In [ ]:
import requests

In [ ]:
#WSJ에 고시되어 있는 데이터들의 기준 날짜가 언제인지 알기 위한 것 (#####크롤링 안돼서 해결 필)

def GET_DATE():
  req = requests.get("https://www.wsj.com/market-data/quotes/bond/BX/TMUBMUSD30Y?mod=md_bond_overview_quote")
  html=req.text
  soup=BeautifulSoup(html,'html.parser')
  data=soup.find("span",class_="timestamp ")
  if data:
        date = data.text
        date = datetime.datetime.strptime(date, "%m/%d/%y").date()
        return date
  else:
        print("The specified element was not found on the webpage.")
        return None

In [ ]:
def GET_QUOTE(reference_date):
  tenors=['01M','03M','06M','01Y','02Y','03Y','05Y','07Y','10Y','30Y']

  #Create Empty Lists
  maturities = []
  days = []
  prices = []
  coupons = []

  #Get Market Information
  for i, tenor in enumerate(tenors):
    req=requests.get("https://quotes.wsj.com/bond/BX/TMUBMUSD" + tenors + "?mod=md_bond_overview_quote")
    html = req.text
    soup = BeautifulSoup(html,'html.parser')

    #price
    if i<=3:
      data_src = soup.find("span",id="quote_val")
      price = data_src.text
      price = float(price[:-1])
    else:
      data_src = soup.find("span",id="price_quote_val")
      price=data_src.text
      price=price.split()
      price1=float(price[0])
      price=price[1].split('/')
      price2=float(price[0])
      price3=float(price[1])
      price=price1+(price2/price3)

    data_src2 = soup.find_all("span", class_="data_data")

    #Coupon
    coupon=data_src2[2].test
    if coupon !='':
      coupon = float(coupon[:-1])
    else:
      coupon = 0.0

    #Maturity Date(만기일)
    maturity = data_src2[3].text
    maturity = datetime.datetime.strptime(maturity, '%m/%d/%y').date()

    #Send to Lists
    days.append((maturity-reference_date).days)
    prices.append(price)
    coupons.append(coupon)
    maturities.append(maturity)

  #Create DataFrame
  df = pd.DataFrame([maturities, days,prices, coupons]).transpose()
  headers=['maturity','days','price','coupon']
  df.columns = headers
  df.set_index('maturity',inplace=True)

  return df

In [ ]:
#Test
ref_date = GET_DATE()
ref_date
#quote = GET_QUOTE(ref_date)
#print(quote)

The specified element was not found on the webpage.


In [ ]:
def TREASURY_CURVE(date, quote):

  #Divide Quotes 데이터 프레임 나누기(무이표채와 이표채를 나누는 방식이 다르기 때문에 이 작업 진행)
  tbill=quote[0:4]
  tbond=quote[4:]

  #Set Evaluation Date(평가일 설정)
  eval_date = ql.Date(date.day,date.month,date.year)
  ql.Settings.instance().evaluationDate=eval_date

  #Set Market Conventions(시장 관행 설정)
  calender = ql.UnitedStates()
  convention = ql.ModifiedFollowing
  day_counter = ql.ActualActual()
  end_of_month = True
  fixing_days = 1
  face_amount = 100
  coupon_frequency = ql.period(ql.Semiannual)

  # Construct Treasury Bill helpers 무이표채에 적용되는 퀀트립 메소드
  bill_helpers = [ql.DepositRateHelper(ql.QuoteHandle(ql.SimpleQuote(r/100.0)),
                                       ql.Period(m,ql.Days),
                                       fixing_days,
                                       calendar,
                                       convention,
                                       end_of_month,
                                       day_counter)
                  for r,m in zip(tbill['price'], tbill['days'])]

  # Construct Treasury Bond Helpers
  bond_helpers = []
  for p,c,m in zip(tbond['price'],tbond['coupon'],tbond['days']):
    termination_date = eval_date + ql.Period(m,ql.Days)
    schedule = ql.Schedule(eval_date,
                           termination_date,
                           coupon_frequency,
                           calendar,
                           convention,
                           convention,
                           ql.DateGeneration.Backward,
                           end_of_month)
    bond_helper = ql.FixedRateBondHelper(ql.QuoteHandle(ql.SimpleQuote(p)),
                                         fixing_days,
                                         face_amount,
                                         schedule,
                                         [c/100.0],
                                         day_counter,
                                         convention)
    bond_helpers.append(bond_helper)

    #Bind Helpers(무이표채와 이표채 Helper 결)

    rate_helper = bill_helpers + bond_helpers

    #Build Curve(헬퍼를 커브 모듈에 태우기, 선형 보간을 사용해서 커브 만듦)
    #선형 보간법은 1차원 직선상에서 두 점의 값이 주어졌을 때 그 사이의 값을 추정하기 위해 직선 거리에 따라 선형적으로 계산하는 방법


    yc_linearzero = ql.PiecewiseLinearZero(eval_date, rate_helper, day_counter)

    return yc_linearzero

    #Discount_Factor
    def DISCOUNT_FACTOR(date, curve):
      date = ql.Date(date.day, date.month, date.year)
      return curve.discount(date)

    #DISCOUNT_FACTOR() 특정일만 알려주면 그

    def DISCOUNT_FACTOR(date, curve):
      date = ql.Date(date.day, date.month, date.year)
      return curve.discount(date)

    #Zero_RATE()

    def ZERO_RATE(date, curve):
      date = ql.Date(date.day, date.month, date.year)
      day_counter = ql.ActualActual()
      compounding = ql.Coumpounded
      freq = ql.Continuous
      zero_rate = curve.zeroRate(date, day_counter, compounding, freq).rate()
      return zeor_rate










In [ ]:
#테스트

ref_date = GET_DATE()
quote = GET_QUOTE(ref_date)
curve = TREASURY_CURVE(ref_date, quote)

quote['discount factor'] = np.nan
quote['zero rate'] = np.nan
curve = TREASURY_CURVE(ref_date, quote)

for date in quote.index:
  quote.loc[date, 'discount factor'] = DISCOUNT_FACTOR(date, curve)
  quote.loc[date, 'zero rate'] = ZERO_RATE(date, curve)

print(quote[['discount facotor', 'zero rate']])

plt.figure(figsize=(16,8))
plt.plot(quote['zero rate'], 'b.-')
plt.title('Zero Curve', loc='center')
plt.xlabel('Maturity')
plt.ylabel('Zero Rate')

plt.figure(figsize=(16,8))
plt.plot(quote['discount factor'],'r.-')
plt.title('Discount Curve', loc='center')
plt.xlabel('Maturity')
plt.ylabel('Discount Factor')